In [2]:
from datetime import date
import json
import requests
import pandas as pd
import numpy as np

# Key Macroeconomic Indicators
https://www.lb.lt/lt/naujausios-ekonomikos-prognozes#ex-1-1 <br>
https://osp.stat.gov.lt/en/pagrindiniai-salies-rodikliai

## BVP Growth
https://stats.oecd.org/index.aspx?lang=en# <br>
Key Short-Term Economic Indicators <br>
Gross domestic product; constant prices, s.a. <br>
Growth on the same period of the previous year <br>
<br>
https://www.seb.lt/infohub-archive?date=any_time&dropdown=any_time&mobile_month=3&mobile_year=2022&month=3&year=2022&field_category%5B662%5D=662

In [59]:
url = f'https://stats.oecd.org/SDMX-JSON/data/KEI/PS+PR+PRINTO01+PRMNTO01+PRCNTO01+SL+SLRTTO01+SLRTCR03+OD+ODCNPI03+CI+LO+LOLITOAA+BS+BSCICP02+LORSGPRT+CS+CSCICP02+LI+LF+LFEMTTTT+LR+LRHUTTTT+LC+LCEAMN01+UL+ULQEUL01+PP+PI+PIEAMP01+CP+CPALTT01+FI+MA+MANMM101+MABMM301+IR+IRSTCI01+IR3TIB01+IRLTLT01+SP+SPASTT01+CCUSMA02+XT+XTEXVA01+XTIMVA01+BP+B6BLTT02+NA+NAEXKP01+NAEXKP02+NAEXKP03+NAEXKP04+NAEXKP06+NAEXKP07.LTU.GP+GY+ST.Q/all?startTime=2016-Q1&endTime={date.today().year}-Q4&dimensionAtObservation=allDimensions'
response = requests.get(url)
json_data = response.json() # This method is convenient when the API returns JSON

subject = pd.DataFrame(json_data['structure']['dimensions']['observation'][0]['values']).reset_index()
subject = subject[subject['name'] == 'Gross domestic product; constant prices, s.a.']

measure = pd.DataFrame(json_data['structure']['dimensions']['observation'][2]['values']).reset_index()
measure = measure[measure['name'] == 'Growth on the same period of the previous year']

gdp_data = pd.DataFrame(json_data['dataSets'][0]['observations']).T.reset_index()
gdp_data = gdp_data[gdp_data['index'].str.startswith(str(f"{subject.index.values[0]}:0:{measure.index.values[0]}:0"))][0]

gdp_data = gdp_data.reset_index(drop = True).reset_index()
gdp_data.columns = ['index', 'gdp_growth']

datos = pd.DataFrame(json_data['structure']['dimensions']['observation'][4]['values']).reset_index()

gdp_data =  pd.concat([gdp_data, datos], axis = 1)
gdp_data = gdp_data[['id', 'gdp_growth']]

gdp_data['year'] = gdp_data['id'].str[:4].astype(int)
gdp_data['quarter'] = gdp_data['id'].str[-1:].astype(int)
gdp_data = gdp_data.drop(columns = ['id']).dropna()

gdp_data.tail(5).round(1)

,gdp_growth,year,quarter
20,1.0,2021,1
21,8.6,2021,2
22,5.3,2021,3
23,5.0,2021,4
24,4.4,2022,1


## Salary growth
https://osp.stat.gov.lt/verslas-lietuvoje-2020/vidutinis-menesinis-darbo-uzmokestis <br>
Average net monthly earnings of employees in the whole economy. EUR <br>
Neto

In [ ]:
def salary_func(year):
    if year == 2016:
        return 8.4
    elif year == 2017:
        return 8.6
    elif year == 2018:
        return 10.0
    elif year == 2019:
        return 8.8
    elif year == 2020:
        return 10.2
    elif year == 2021:
        return 10.5
    elif year == 2022:
        return 11.9
    elif year == 2023:
        return 8.1
    else:
        return 0  
    
f'KMI last updated: {date.today()}'

In [ ]:
# ## Vartotojų kainų pokyčiai, apskaičiuoti pagal vartotojų kainų indeksą
# ## Individualaus vartojimo išlaidų pagal paskirtį klasifikatorius (COICOP) (1991M01 - 2022M06) (Atnaujinta: 2022-07-11)

# url = f'https://osp-rs.stat.gov.lt/rest_json/data/S3R0050_M3060315_1/?startPeriod=2016-01&endPeriod={date.today().year}-12'

# response = requests.get(url)
# json_data = response.json() # This method is convenient when the API returns JSON

# sektorius = pd.DataFrame(json_data['structure']['dimensions']['observation'][0]['values']).reset_index()
# sektorius = sektorius[sektorius['name'] == 'Šalies ūkis su individualiosiomis įmonėmis']

# veikla = pd.DataFrame(json_data['structure']['dimensions']['observation'][1]['values']).reset_index()
# veikla = veikla[veikla['name'] == 'Iš viso pagal ekonomines veiklos rūšis']

# lytis = pd.DataFrame(json_data['structure']['dimensions']['observation'][2]['values']).reset_index()
# lytis = lytis[lytis['name'] == 'Vyrai ir moterys']

# tipas = pd.DataFrame(json_data['structure']['dimensions']['observation'][3]['values']).reset_index()
# tipas = tipas[tipas['name'] == 'Neto']

# salary_data = pd.DataFrame(json_data['dataSets'][0]['observations']).T.reset_index()
# salary_data = salary_data[salary_data['index'].str.startswith(str(f"{sektorius.index.values[0]}:{veikla.index.values[0]}:{lytis.index.values[0]}:{tipas.index.values[0]}:"))][0]

# salary_data = salary_data.reset_index(drop = True).reset_index()
# salary_data.columns = ['index', 'Salary growth']

# datos = pd.DataFrame(json_data['structure']['dimensions']['observation'][5]['values']).reset_index()

# salary_data =  pd.concat([salary_data, datos], axis = 1)
# salary_data = salary_data[['id', 'Salary growth']]

# salary_data['year'] = salary_data['id'].str[:4].astype(int)
# salary_data['quarter'] = salary_data['id'].str[-1:].astype(int)
# salary_data.drop(columns = ['id'], inplace = True)

# salary_data = salary_data.sort_values(['year', 'quarter'])

# salary_data = pd.concat([salary_data.rename({'Salary growth':'curr_value'}, axis = 'columns'),
#                      salary_data.shift(4).rename({'Salary growth':'prev_value'}, axis = 'columns')], axis = 1)

# salary_data['Salary growth'] = (salary_data['curr_value'] / salary_data['prev_value'] - 1) * 100
# salary_data = salary_data.loc[:,~salary_data.columns.duplicated()].dropna()

# salary_data = salary_data[['year', 'quarter', 'Salary growth']]
# salary_data.head(5)

## Export & Import Growth
https://osp.stat.gov.lt/statistiniu-rodikliu-analize#/ <br>
Prekių ir paslaugų eksportas, palyginamosiomis kainomis (grandininio susiejimo metodu) <br>
Prekių ir paslaugų importas, palyginamosiomis kainomis (grandininio susiejimo metodu)

In [ ]:
def export_func(year):
    if year == 2016:
        return 4.9
    elif year == 2017:
        return 13.5
    elif year == 2018:
        return 6.8
    elif year == 2019:
        return 9.9
    elif year == 2020:
        return 0.4
    elif year == 2021:
        return 15.9
    elif year == 2022:
        return 7.4
    elif year == 2023:
        return 2.9
    else:
        return 0      
# '==============================='
def import_func(year):
    if year == 2016:
        return 4.0
    elif year == 2017:
        return 11.1
    elif year == 2018:
        return 6.0
    elif year == 2019:
        return 6.1
    elif year == 2020:
        return -4.4
    elif year == 2021:
        return 18.7
    elif year == 2022:
        return 9.8
    elif year == 2023:
        return 2.8
    else:
        return 0   
    
f'KMI last updated: {date.today()}'

## Inflation Growth
https://tradingeconomics.com/lithuania/forecast <br>
Inflation Rate (%)

In [12]:
## Vartotojų kainų pokyčiai, apskaičiuoti pagal vartotojų kainų indeksą
## Individualaus vartojimo išlaidų pagal paskirtį klasifikatorius (COICOP) (1991M01 - 2022M06) (Atnaujinta: 2022-07-11)

url = f'https://osp-rs.stat.gov.lt/rest_json/data/S7R121_M2020103/?startPeriod=2017-01&endPeriod={date.today().year}-12'

response = requests.get(url)
json_data = response.json() # This method is convenient when the API returns JSON

klasifikatorius = pd.DataFrame(json_data['structure']['dimensions']['observation'][0]['values']).reset_index()
klasifikatorius = klasifikatorius[klasifikatorius['id'] == '00']

lyginimas = pd.DataFrame(json_data['structure']['dimensions']['observation'][1]['values']).reset_index()
lyginimas = lyginimas[lyginimas['id'] == 'palyg_pm']

cpi_data = pd.DataFrame(json_data['dataSets'][0]['observations']).T.reset_index()
cpi_data = cpi_data[cpi_data['index'].str.startswith(str(f"{klasifikatorius.index.values[0]}:{lyginimas.index.values[0]}:0:"))][0]

cpi_data = cpi_data.reset_index(drop = True).reset_index()
cpi_data.columns = ['index', 'inflation_growth']

datos = pd.DataFrame(json_data['structure']['dimensions']['observation'][3]['values']).reset_index()

cpi_data =  pd.concat([cpi_data, datos], axis = 1)
cpi_data = cpi_data[['id', 'inflation_growth']]

cpi_data['year'] = cpi_data['id'].str[:4].astype(int)
cpi_data['month'] = cpi_data['id'].str[-2:].astype(int)
cpi_data['quarter'] = [int(np.ceil(x / 3))  for x in cpi_data['month']]
cpi_data.drop(columns = ['id'], inplace = True)

cpi_data.tail(5)

,inflation_growth,year,month,quarter
61,14.2,2022,2,1
62,15.7,2022,3,1
63,16.8,2022,4,2
64,18.9,2022,5,2
65,21.0,2022,6,2


## Unemployed & Employment Growth

In [ ]:
def unemployed_func(year):
    if year == 2016:
        return 7.9
    elif year == 2017:
        return 7.1
    elif year == 2018:
        return 6.1
    elif year == 2019:
        return 6.3
    elif year == 2020:
        return 8.5
    elif year == 2021:
        return 7.1
    elif year == 2022:
        return 7.3
    elif year == 2023:
        return 7.2
    else:
        return 0      
# '================================================================================='
def employed_func(year):
    if year == 2016:
        return 2.0
    elif year == 2017:
        return -0.5
    elif year == 2018:
        return 1.5
    elif year == 2019:
        return 0.3
    elif year == 2020:
        return -1.5
    elif year == 2021:
        return 0.8
    elif year == 2022:
        return 0.6
    elif year == 2023:
        return 0.3
    else:
        return 0    
    
f'KMI last updated: {date.today()}'

In [ ]:
# ## Nedarbo lygis
# ## Amžius (tikslinės amžiaus grupės) | Gyvenamoji vietovė | Lytis (1998K2 - 2022K1) (Atnaujinta: 2022-05-13)

# url = f'https://osp-rs.stat.gov.lt/rest_json/data/S3R347_M3030101_1/?startPeriod=2017-01&endPeriod={date.today().year}-12'

# response = requests.get(url)
# json_data = response.json() # This method is convenient when the API returns JSON

# amzius = pd.DataFrame(json_data['structure']['dimensions']['observation'][0]['values']).reset_index()
# amzius = amzius[amzius['name'] == 'Iš viso pagal amžiaus grupes']

# gyvenama_vieta = pd.DataFrame(json_data['structure']['dimensions']['observation'][1]['values']).reset_index()
# gyvenama_vieta = gyvenama_vieta[gyvenama_vieta['name'] == 'Miestas ir kaimas']

# lytis = pd.DataFrame(json_data['structure']['dimensions']['observation'][2]['values']).reset_index()
# lytis = lytis[lytis['name'] == 'Vyrai ir moterys']

# unemployment_data = pd.DataFrame(json_data['dataSets'][0]['observations']).T.reset_index()
# unemployment_data = unemployment_data[unemployment_data['index'].str.startswith(str(f"{amzius.index.values[0]}:{gyvenama_vieta.index.values[0]}:{lytis.index.values[0]}:0:"))][0]

# unemployment_data = unemployment_data.reset_index(drop = True).reset_index()
# unemployment_data.columns = ['index', 'Unemployment level']

# datos = pd.DataFrame(json_data['structure']['dimensions']['observation'][4]['values']).reset_index()

# unemployment_data =  pd.concat([unemployment_data, datos], axis = 1)
# unemployment_data = unemployment_data[['id', 'Unemployment level']]

# unemployment_data['year'] = unemployment_data['id'].str[:4].astype(int)
# unemployment_data['quarter'] = unemployment_data['id'].str[-1:].astype(int)

# unemployment_data.head(5)

In [ ]:
# ## Nedarbo lygis
# ## Amžius (tikslinės amžiaus grupės) | Gyvenamoji vietovė | Lytis (1998K2 - 2022K1) (Atnaujinta: 2022-05-13)

# url = f'https://osp-rs.stat.gov.lt/rest_json/data/S3R532_M3030101_2/?startPeriod=2016-01&endPeriod={date.today().year}-12'

# response = requests.get(url)
# json_data = response.json() # This method is convenient when the API returns JSON

# amzius = pd.DataFrame(json_data['structure']['dimensions']['observation'][0]['values']).reset_index()
# amzius = amzius[amzius['name'] == 'Iš viso pagal amžiaus grupes']

# gyvenama_vieta = pd.DataFrame(json_data['structure']['dimensions']['observation'][1]['values']).reset_index()
# gyvenama_vieta = gyvenama_vieta[gyvenama_vieta['name'] == 'Miestas ir kaimas']

# lytis = pd.DataFrame(json_data['structure']['dimensions']['observation'][2]['values']).reset_index()
# lytis = lytis[lytis['name'] == 'Vyrai ir moterys']

# employment_data = pd.DataFrame(json_data['dataSets'][0]['observations']).T.reset_index()
# employment_data = employment_data[employment_data['index'].str.startswith(str(f"{amzius.index.values[0]}:{gyvenama_vieta.index.values[0]}:{lytis.index.values[0]}:0:"))][0]

# employment_data = employment_data.reset_index(drop = True).reset_index()
# employment_data.columns = ['index', 'Employment level']

# datos = pd.DataFrame(json_data['structure']['dimensions']['observation'][4]['values']).reset_index()

# employment_data =  pd.concat([employment_data, datos], axis = 1)
# employment_data = employment_data[['id', 'Employment level']]

# employment_data['year'] = employment_data['id'].str[:4].astype(int)
# employment_data['quarter'] = employment_data['id'].str[-1:].astype(int)

# employment_data = employment_data.sort_values(['year', 'quarter'])

# employment_data = pd.concat([employment_data.rename({'Employment level':'curr_value'}, axis = 'columns'),
#                      employment_data.shift(4).rename({'Employment level':'prev_value'}, axis = 'columns')], axis = 1)

# employment_data['Employment level'] = (employment_data['curr_value'] / employment_data['prev_value'] - 1) * 100
# employment_data = employment_data.loc[:,~employment_data.columns.duplicated()].dropna()

# employment_data = employment_data.merge(unemployment_data, on = ['year', 'quarter'])
# employment_data = employment_data[['year', 'quarter', 'Employment level', 'Unemployment level']]

# employment_data.head(5)

## Consumption expenditure
BVP išlaidų metodu, palyginamosiomis kainomis (grandininio susiejimo metodu) <br>
Namų ūkių ir NPI galutinio vartojimo išlaidos <br>
Valdžios sektoriaus galutinio vartojimo išlaidos

In [ ]:
def privat_consump_func(year):
    if year == 2016:
        return 4.1
    elif year == 2017:
        return 3.6
    elif year == 2018:
        return 3.6
    elif year == 2019:
        return 3.1
    elif year == 2020:
        return -2.1
    elif year == 2021:
        return 7.3
    elif year == 2022:
        return 5.4
    elif year == 2023:
        return 3.8
    else:
        return 0      
# '================================================================================='
def govern_consump_func(year):
    if year == 2016:
        return 0.1
    elif year == 2017:
        return -0.3
    elif year == 2018:
        return 0.2
    elif year == 2019:
        return -0.3
    elif year == 2020:
        return -0.4
    elif year == 2021:
        return 0.5
    elif year == 2022:
        return 1.5
    elif year == 2023:
        return -1.3
    else:
        return 0    
    
f'KMI last updated: {date.today()}'